In [ ]:
# Enter details of your AML workspace
subscription_id = "b3b0e63c-e8fd-4f5c-bab9-1ed82844ef1f"
resource_group = "jameschao"
workspace = "jameschao"
compute_name = "jameschao1"

# Enter data details
data_version = "1"
data_path = "data-forecasting/"
train_data_path = data_path + "train/"
test_data_path = data_path + "test/"

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace,
)

# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)

In [ ]:
# Access data
import mltable
import pandas as pd

# 1. Convert data from csv to parquet format

# train_df = pd.read_csv(train_data_path + "train.txt")
# train_df.to_parquet(train_data_path + "train.parquet")
# test_df = pd.read_csv(test_data_path + "test.txt")
# test_df.to_parquet(test_data_path + "test.parquet")

# 2. Create MLTable files

# mltable_train_contents = f"""
# $schema: http://azureml/sdk-2-0/MLTable.json
# type: mltable
# paths:
#   - file: ./train.parquet
# transformations:
#   - read_parquet
# """

# mltable_test_contents = f"""
# $schema: http://azureml/sdk-2-0/MLTable.json
# type: mltable
# paths:
#   - file: ./test.parquet
# transformations:
#   - read_parquet
# """

# mltable_train_filename = train_data_path + "MLTable"
# mltable_test_filename = test_data_path + "MLTable"

# with open(mltable_train_filename, "w") as f:
#     f.write(mltable_train_contents)

# with open(mltable_test_filename, "w") as g:
#     g.write(mltable_test_contents)

# 3. Load data to preview

train_df = mltable.load(train_data_path).to_pandas_dataframe()
test_df = mltable.load(test_data_path).to_pandas_dataframe()
assert len(test_df.index) <= 5000
print("TRAIN DTYPES:\n", train_df.dtypes, "\n\n")
print("TEST DTYPES:\n", test_df.dtypes, "\n\n")
test_df

In [ ]:
# 4. If the datatype of your date column is note datetime64[ns], it must be converted.

# train_df["datetime"] = train_df["datetime"].astype("datetime64[ns]")
# test_df["datetime"] = test_df["datetime"].astype("datetime64[ns]")

# train_df.to_parquet(train_data_path + "train.parquet")
# test_df.to_parquet(test_data_path + "test.parquet")

# 5. If no time_series_id column (such as group id), create one group for the entire timeseries.

# train_df["group_id"] = 1.0
# test_df["group_id"] = 1.0

# train_df.to_parquet(train_data_path + "train.parquet")
# test_df.to_parquet(test_data_path + "test.parquet")

In [ ]:
import matplotlib.pyplot as plt

print(
    f"Train dates : {train_df.index.min()} --- {train_df.index.max()}  (n={len(train_df)})"
)
print(
    f"Test dates  : {test_df.index.min()} --- {test_df.index.max()}  (n={len(test_df)})"
)


fig, ax = plt.subplots(figsize=(6, 2.5))
train_df["demand"].plot(ax=ax, label="train")
test_df["demand"].plot(ax=ax, label="test")
ax.legend();

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_train_data = "forecasting_energy_train_mltable"
input_test_data = "forecasting_energy_test_mltable"

train_data = Data(
    path=train_data_path,
    type=AssetTypes.MLTABLE,
    description="Forecasting NYC energy example training data",
    name=input_train_data,
    version=data_version,
)
ml_client.data.create_or_update(train_data)

test_data = Data(
    path=test_data_path,
    type=AssetTypes.MLTABLE,
    description="Forecasting NYC energy example test data",
    name=input_test_data,
    version=data_version,
)
ml_client.data.create_or_update(test_data)